In [ ]:
import pandas as pd
import plotly.express as px
import numpy as np
import plotly.graph_objects as go

## Doden bij nucleare energy

In [ ]:
df = pd.read_csv('nuclear_data/rates_death_from_energy_production_per_twh.csv')
df = df.sort_values(by='Deaths per TWh of electricity production', ascending=False)
df.head()

In [ ]:
fig = px.bar(
    df,
    x='Entity',
    y='Deaths per TWh of electricity production',
    color='Deaths per TWh of electricity production',
    color_continuous_scale='RdYlGn_r'  # correcte naam
)

fig.update_layout(
    title="Death-rate per TWh produced per energy source",
    xaxis_title="Energy production method",
    yaxis_title="Deaths per TWh",
    coloraxis_showscale=False,
)

fig.show()

In [ ]:
df = pd.read_csv('impacts-of-energy-sources.csv')

df = df.drop(['Agricultural land use', 'Urban land use', 'Death rates', 'Year'], axis=1)

df = df.rename(columns={'Greenhouse gas emissions': 'Greenhouse gas emissions (kg CO2 eq.)', 
                        'Freshwater eutrophication': 'Freshwater eutrophication (kg P eq.)', 
                        'Ionising radiation':'Ionising radiation (kBq 235U eq)', 
                        'Water use':'Water use (m3)',
                        'Metal and mineral requirements':'Metal and mineral requirements (kg Sb eq.)',
                        'Non-carcinogenic toxicity':'Non-carcinogenic toxicity (CTUh)',
                        'Carcinogenic toxicity':'Carcinogenic toxicity (CTUh)',
                        'Total land use':'Total land use (points)',
                        'Uranium':'Uranium (tonnes)'})

df['Entity'] = df['Entity'].replace('Coal with carbon capture (CCS)', 'Coal')

df['Entity'] = df['Entity'].replace('Concentrating solar (tower)', 'Concentrating Solar')
df['Entity'] = df['Entity'].replace('Concentrating solar (trough)', 'Concentrating Solar')

df['Entity'] = df['Entity'].replace('Solar PV, cadmium (roof)', 'Solar PV')
df['Entity'] = df['Entity'].replace('Solar PV, cadmium (on-ground)', 'Solar PV')
df['Entity'] = df['Entity'].replace('Solar PV, silicon (on-ground)', 'Solar PV')
df['Entity'] = df['Entity'].replace('Solar PV, silicon (roof)', 'Solar PV')

df['Entity'] = df['Entity'].replace('Gas with carbon capture (CCS)', 'Gas')

df['Entity'] = df['Entity'].replace('Offshore wind', 'Wind')
df['Entity'] = df['Entity'].replace('Onshore wind', 'Wind')

# Groepeer en sommeer

df_combined = df.drop(columns=['Aluminium', 'Chromium', 'Cobalt', 'Copper', 'Manganese', 'Molybdenum', 'Nickel', 'Silicon', 'Zinc'])
df_combined['Human toxicity (CTUh)'] = df_combined['Non-carcinogenic toxicity (CTUh)'] + df_combined['Carcinogenic toxicity (CTUh)']
df_combined = df_combined.drop(columns=['Non-carcinogenic toxicity (CTUh)', 'Carcinogenic toxicity (CTUh)' ])
df_combined = df_combined[['Entity', 'Greenhouse gas emissions (kg CO2 eq.)', 
                           'Freshwater eutrophication (kg P eq.)', 
                           'Ionising radiation (kBq 235U eq)', 
                           'Water use (m3)', 
                           'Total land use (points)', 
                           'Human toxicity (CTUh)', 
                           'Metal and mineral requirements (kg Sb eq.)', 
                           'Uranium (tonnes)' ]]
df_combined = df_combined.groupby('Entity', as_index=False).sum()

In [ ]:
# Create a bar chart for each column (except Entity)
for column in df_combined.columns[1:]:
    fig = px.bar(df_combined, 
                 x='Entity', 
                 y=column,
                 title=f'{column} by Energy Source per the delivery of 1 kWh of electricity',
                 labels={'Entity': 'Energy Source', column: column},
                 color='Entity')
    
    # Improve layout for better readability
    fig.update_layout(
        xaxis_title='Energy Source',
        yaxis_title=column,
        height=600,
        width=800,
        title_font=dict(size=15),
        title_x=0.5
    )
    
    # Show the plot
    fig.show()

In [ ]:
import pandas as pd

df = pd.read_csv("nuclear_data/levelised_cost_of_electricity.csv")
print(df.head())


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv("nuclear_data/levelised_cost_of_electricity.csv").drop(index=0)
df = df.rename(columns={
    'Capital costs': 'Capital',
    'O&M': 'OM',
    'Fuel (th)': 'Fuel_th',
    'Fuel (el)': 'Fuel_el',
    'Carbon': 'Carbon',
    'CHP heat revenues': 'CHP_revenue',
    'LCOE': 'LCOE',
    'Category': 'Category',
    'Country': 'Country'
})
df[['LCOE']] = df[['LCOE']].apply(pd.to_numeric, errors='coerce')
df = df.dropna(subset=['Category', 'LCOE'])

nuclear = df[df['Category'] == 'Nuclear']
nuclear_mean = nuclear['LCOE'].mean()

competitors = ['Wind', 'Solar', 'Gas', 'Coal']
comp_means = {}
savings = {}

for comp in competitors:
    mean = df[df['Category'] == comp]['LCOE'].mean()
    comp_means[comp] = mean
    savings[comp] = ((mean - nuclear_mean) / mean) * 100

labels = ['Nuclear'] + competitors
values = [nuclear_mean] + [comp_means[c] for c in competitors]

plt.figure(figsize=(8, 6))
bars = plt.bar(labels, values, color=['green'] + ['gray']*len(competitors))
plt.ylabel("Average LCOE (USD/MWh)")
plt.title("Average LCOE by Energy Source")
plt.grid(False)
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval + 1, f"${yval:.1f}", ha='center')
plt.ylim(0, max(values) + 20)
plt.tight_layout()
plt.show()

plt.figure(figsize=(8, 6))
bars2 = plt.bar(savings.keys(), savings.values(), color='green')
plt.ylabel("Percent Cheaper Than Competitor")
plt.title("Nuclear Cost Advantage Over Other Technologies")
plt.grid(False)

for bar in bars2:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval + 1, f"{yval:.1f}%", ha='center')
plt.ylim(0, max(savings.values()) + 10)
plt.axhline(0, color='black', linewidth=0.8)
plt.tight_layout()
plt.show()



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

df = pd.read_csv("nuclear_data/levelised_cost_of_electricity.csv").drop(index=0)

df = df.rename(columns={
    'Capital costs': 'Capital',
    'O&M': 'OM',
    'Fuel (th)': 'Fuel_th',
    'Fuel (el)': 'Fuel_el',
    'Carbon': 'Carbon',
    'CHP heat revenues': 'CHP_revenue',
    'LCOE': 'LCOE',
    'Category': 'Category',
    'Country': 'Country'
})

cols = ['Capital', 'OM', 'Fuel_th', 'Fuel_el', 'Carbon', 'CHP_revenue', 'LCOE']
df[cols] = df[cols].apply(pd.to_numeric, errors='coerce')

df = df.dropna(subset=['Category', 'LCOE'])

sns.set(style="whitegrid")

plt.figure(figsize=(10, 6))
avg_lcoe = df.groupby('Category')['LCOE'].mean().sort_values()

n_colors = len(avg_lcoe)
palette = sns.color_palette("RdYlGn_r", n_colors) 

sns.barplot(x=avg_lcoe.values, y=avg_lcoe.index, palette=palette)
plt.title("Average LCOE by Technology Category")
plt.xlabel("LCOE (USD/MWh)")
plt.tight_layout()
plt.show()


